In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import risk_kit as rk
from pricing_models import LevelPaymentMortgage, PassThroughMBS, StockPricing, SwapsPricing, OptionsPricing
import scipy.optimize as opt

In each of the following questions monthly mortgage coupon rates should be calculated by simply dividing the annual rate by 12. You should also assume that all of the securities pay monthly. You should also divide annual interest rates by 12 to get the corresponding monthly rate and assume monthly compounding when computing present values.

## Level-Payment Mortgage

### Question 1

Compute the monthly payment on a 30-year level payment mortgage assuming an annual mortgage rate of 5% and an initial mortgage principal of 400,000 usd.

In [2]:
T = 30
r = 0.05
P = 400000

loan = LevelPaymentMortgage(P, r, T)
round(loan.monthly_payment, 2)

2147.29

## Mortgage Pass-Throughs

Consider a 400 million usd pass-through MBS that has just been created (so the 'seasoning' of the pass-through is equal to 0). The underlying pool of mortgages each has a maturity of 20 years and an annual mortgage coupon rate of 6%. The pass-through rate of the mortgage pool is 5%. 

In [3]:
P = 400
T = 20
rate_recieved = 0.06
rate_paid = 0.05
PSA = 100

mortgage_pool = PassThroughMBS(P=P,
                              T=T,
                              loan_r=rate_recieved,
                              pass_r=rate_paid,
                              PSA=PSA)

mortgage_pool.data.head()

d:\techlent\lecture\Financial Engineering and Risk Management\Introduction-to-Financial-Engineering-and-Risk-Management\pricing_models.py:1141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, pd.DataFrame([current_values])], ignore_index=True)


,Total Payment Received,Principal Received,Interest Received,Total Amount Paid,Principal Paid,Interest Paid,Earning,Prepayment Rate,Prepayment Amount,Total OutStanding Amount
1,2.865724,0.865724,2.000000,2.598974,0.932308,1.666667,0.266750,0.000167,0.066583,399.067692
2,2.865246,0.869908,1.995338,2.665666,1.002884,1.662782,0.199580,0.000334,0.132977,398.064808
3,2.864289,0.873965,1.990324,2.731712,1.073109,1.658603,0.132577,0.000501,0.199144,396.991699
4,2.862853,0.877895,1.984958,2.797076,1.142944,1.654132,0.065777,0.000669,0.265049,395.848755
5,2.860938,0.881694,1.979244,2.861721,1.212351,1.649370,-0.000784,0.000837,0.330657,394.636404


### Question 2

Assuming a prepayment multiplier of 100 PSA. What is the total amount of interest paid to the pass-through investors?

In [4]:
round(mortgage_pool.data['Interest Paid'].sum(), 2)

171.18

### Question 3

Referring to the same mortgage pass-through of the previous question, what is the total amount of the prepayments?

In [5]:
round(mortgage_pool.data['Prepayment Amount'].sum(), 2)

181.09

### Question 4

Referring to the same mortgage pass-through of the previous question, what is the total amount of the prepayments if the rate of prepayments increases to 200 PSA?

In [6]:
PSA = 200

mortgage_pool = PassThroughMBS(P=P,
                              T=T,
                              loan_r=rate_recieved,
                              pass_r=rate_paid,
                              PSA=PSA)
round(mortgage_pool.data['Prepayment Amount'].sum(), 2)

d:\techlent\lecture\Financial Engineering and Risk Management\Introduction-to-Financial-Engineering-and-Risk-Management\pricing_models.py:1141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, pd.DataFrame([current_values])], ignore_index=True)


268.15

## Principal-Only MBS and Interest-Only MBS

### Question 1 (Pricing swaps using term structure model)

Consider an n = 10-period binomial model for the short-rate, $r_{i,j}$. The lattice parameters are: 
$r_{0,0}$=5%, u=1.1, d=0.9 and q=1−q=1/2.

Compute the initial value of a forward-starting swap that begins at t=1, with maturity t = 10 and a fixed rate of 4.5%. 

(The first payment then takes place at t = 2 and the final payment takes place at t = 11 as we are assuming, as usual, that payments take place in arrears.) You should assume a swap notional of 1 million and assume that you receive floating and pay fixed.)

In [7]:
n = 10
r00 = 0.05
u = 1.1
d = 0.9
q = 0.5

In [8]:
rate_model = StockPricing(n, r00, u, d)

In [9]:
n = 11
fixed_rate = 0.045
start = 2
is_long = 1
N = 1000000

swap_pricing = SwapsPricing(n, q, fixed_rate, start_time=2, is_long=is_long, r=rate_model)
round(swap_pricing.price * N)

33374

### Question 2 (Pricing swaptions using term structure model)

Compute the initial price of a swaption that matures at time t=5 and has a strike of 0. The underlying swap is the same swap as described in the previous question with a notional of 1 million. To be clear, you should assume that if the swaption is exercised at t=5 then the owner of the swaption will receive all cash-flows from the underlying swap from times t=6 to t=11 inclusive. (The swaption strike of 0 should also not be confused with the fixed rate of 4.5% on the underlying swap.)

In [10]:
# Assuming the European Call Option
n = 5
strike = 0
N = 1000000

option_pricing = OptionsPricing(n, swap_pricing, rate_model, q, strike)
round(option_pricing.price * N)

26311

### Question 3 (Calibration of term structure model)

Refer to the material on defaultable bonds and credit-default swaps (CDS).

The true price of 5 different defaultable coupon paying bonds with non-zero recovery are specified in  worksheet Calibration in the workbook 
Assignment5_cds.xlsx. The interest rate is r=5% per annum. Calibrate the six month hazard rates A6 to A16 by minimizing the SumError ensuring that the term structure of hazard rates are non-decreasing. You can model the non-decreasing hazard rates by adding constraints of the form A6≤A7,…,A15≤A16. Report the hazard rate at time 0 as a percentage. 

Answer - 1.84

### Question 4 Question 4 (Pricing Credit Default Swaps)

Refer to the material on defaultable bonds and credit-default swaps (CDS).

Modify the data on the CDS pricing worksheet in the workbook bonds_and_cds.xlsx to compute a par spread in basis points for a 5yr CDS with notional principal N=10 million assuming that the expected recovery rate R=25%, the 3-month hazard rate is a flat 1%, and the interest rate is 5% per annum.

Answer - 301.51

### Question 5 (Principal-only and Interest-only MBS - Part A)

Background: In each of the following questions, monthly mortgage coupon rates should be calculated by simply dividing the annual rate by 12. You should also assume that all of the securities pay monthly. Also, you should divide annual interest rates by 12 to get the corresponding monthly rate and assume monthly compounding when computing present values.

Suppose we construct principal-only (PO) and interest-only (IO) mortgage-backed securities (MBS) using the mortgage pass-through of the previous questions. Assume a prepayment multiplier of 100 PSA. 

In [11]:
PSA = 100

mortgage_pool = PassThroughMBS(P=P,
                              T=T,
                              loan_r=rate_recieved,
                              pass_r=rate_paid,
                              PSA=PSA)

d:\techlent\lecture\Financial Engineering and Risk Management\Introduction-to-Financial-Engineering-and-Risk-Management\pricing_models.py:1141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, pd.DataFrame([current_values])], ignore_index=True)


What is the present value of the PO MBS if we use an annual risk-free rate of 4.5% to value the cash-flows?

In [12]:
cash_flows_po = mortgage_pool.data['Principal Paid']
rate = 0.045 / 12
pv = float(rk.present_value(flows=cash_flows_po, discount_rate=rate))
round(pv, 2)

C:\Users\x_tia\AppData\Local\Temp\ipykernel_1572\2864489323.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pv = float(rk.present_value(flows=cash_flows_po, discount_rate=rate))


280.1

### Question (Principal-only and Interest-only MBS - Part B)

Referring to the previous question, what is the value of the IO MBS?

In [13]:
cash_flows_io = mortgage_pool.data['Interest Paid']
rate = 0.045 / 12
pv_r45 = float(rk.present_value(flows=cash_flows_io, discount_rate=rate))
round(pv_r45, 2)

C:\Users\x_tia\AppData\Local\Temp\ipykernel_1572\3442375128.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pv_r45 = float(rk.present_value(flows=cash_flows_io, discount_rate=rate))


133.23

### Question 7 (Principal-only and Interest-only MBS - Part C)

Referring to the previous question, what is the average life of the IO MBS?

In [14]:
round(rk.duration(cash_flows_io) / 12, 2)

6.01

### Question 8 (Principal-only and Interest-only MBS - Part D)

Suppose now that you purchased the IO MBS of the previous question and that the price you paid was the same price that you calculated in the previous question. The risk-free interest rate suddenly changes from 4.5% to 3.5%. Everything else stays the same. How much money have you made or lost on your investment?

In [15]:
cash_flows_io = mortgage_pool.data['Interest Paid']
rate = 0.035 / 12
pv_r35 = float(rk.present_value(flows=cash_flows_io, discount_rate=rate))
round(pv_r35 - pv_r45, 2)

C:\Users\x_tia\AppData\Local\Temp\ipykernel_1572\1080693897.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pv_r35 = float(rk.present_value(flows=cash_flows_io, discount_rate=rate))


7.17

### Question 9 (Principal-only and Interest-only MBS - Part E)

Referring to the previous question, suppose the risk-free interest rate suddenly changes from 4.5% to 3.5% and that the pre-payment multiplier changes from 100 PSA to 150 PSA. How much money have you made or lost on your investment in the IO MBS?

In [16]:
PSA = 150
rate_new = 0.035/12

mortgage_pool_new = PassThroughMBS(P=P,
                                  T=T,
                                  loan_r=rate_recieved,
                                  pass_r=rate_paid,
                                  PSA=PSA)
cash_flows_io_new = mortgage_pool_new.data['Interest Paid']
pv_new = float(rk.present_value(flows=cash_flows_io_new, discount_rate=rate_new))
round(pv_new - pv_r45, 2)

d:\techlent\lecture\Financial Engineering and Risk Management\Introduction-to-Financial-Engineering-and-Risk-Management\pricing_models.py:1141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, pd.DataFrame([current_values])], ignore_index=True)
C:\Users\x_tia\AppData\Local\Temp\ipykernel_1572\3281036712.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pv_new = float(rk.present_value(flows=cash_flows_io_new, discount_rate=rate_new))


-9.58